In [1]:
import sys
import os

import torch

sys.path.append(os.path.abspath("/Users/liuzhe/github/DPL_study/pys"))
#from single_env_from_csv import *


In [3]:
#e = SingleEnv('/Users/liuzhe/github/DPL_study/data/snap.csv', 'rb')
one_long = torch.tensor([1, 0, 0, 0])
one_short = torch.tensor([0, 1, 0, 0])
one_close = torch.tensor([0, 0, 1, 0])
one_hold = torch.tensor([0, 0, 0, 1])
print(3)





3


In [3]:
e.render()
e.data

timeline: 31, net: 200810.0, cash: 199990, profile: 820.0, direction: 1, in_price: 6894.0, now_price: 6976.0


,day,time,price
0,20200203,09:00:00,6895.0
1,20200203,09:00:01,6894.0
2,20200203,09:00:02,6886.0
3,20200203,09:00:03,6894.0
4,20200203,09:00:04,6888.0
...,...,...,...
17064,20200203,15:14:35,-1.0
17065,20200203,15:14:36,-1.0
17066,20200203,15:14:37,-1.0
17067,20200203,15:14:38,-1.0


In [4]:
import torch
import torch.nn.functional as F
d1 = torch.nn.Linear(100, 50) # y = A(100x50) * x + b,
input = torch.rand(140, 100)
out = d1(input)
out.shape


d2 = torch.nn.Linear(4, 5)
input_matrix = torch.tensor([[1.0, 0.0, 0.0, 0.0]])
F.relu(d2(input_matrix))

d3 = torch.nn.Conv1d(4, 1, 3)

time_input = torch.tensor([[
    [323.0, 324.0, 323.0, 325.0,], # tick
    [310.0, 310.0, 310.0, 311.0,], # min
    [300.0, 300.0, 300.0, 300.0],  # hour
    [300.0, 300.0, 300.0, 300.0],  # hour
]])

d4 = torch.nn.Conv2d(4, 1, 3)

d3(time_input)


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([[[194.5166, 194.1873]]], grad_fn=<SqueezeBackward1>)

In [7]:
metric_num = 15
step_num = 20
time_scales = len(['tick', 'min', '5min', '15min', 'hour', 'day'])
action_count = len(['b1', 's1', 'h', 'c'])

fake_tick = torch.randn((time_scales, step_num, metric_num)) #
fake_tick



tensor([[[ 1.0032e+00,  1.5410e+00, -1.4889e+00,  ...,  5.6558e-01,
           1.1138e+00, -1.4912e-01],
         [ 1.3747e+00, -7.8695e-01,  3.1138e-01,  ..., -1.2692e+00,
           2.9460e-01, -5.9962e-01],
         [ 1.3950e+00, -1.8491e-01, -8.0086e-01,  ..., -1.2672e-01,
           8.3042e-02,  1.0856e+00],
         ...,
         [-1.1643e+00, -1.2447e+00,  1.4328e-01,  ..., -6.3540e-01,
          -1.4191e-01, -6.9446e-02],
         [-9.4009e-01, -1.1390e+00,  1.1633e+00,  ...,  2.2204e+00,
           8.5232e-01, -5.0902e-01],
         [-1.2908e+00, -1.2359e+00,  1.5395e+00,  ...,  1.7412e-01,
           1.1299e+00,  9.7528e-01]],

        [[ 2.7921e-01,  3.1281e-01, -3.4398e-01,  ..., -7.2510e-02,
           3.3203e-01, -7.1003e-01],
         [ 3.9266e-01,  5.8563e-01, -2.7171e-01,  ..., -1.0725e+00,
          -1.2559e+00, -4.1624e-01],
         [-1.1150e+00, -6.9392e-01, -1.8016e+00,  ...,  3.9284e-01,
          -4.6485e-01,  6.6194e-01],
         ...,
         [ 2.1654e-01,  6

In [16]:
conv_out_rows = 10

mod = torch.nn.Sequential(
    torch.nn.Conv1d(step_num, conv_out_rows, 4),
    torch.nn.Flatten(end_dim=1)
)
out = mod(fake_tick)
out.size()

torch.Size([6, 10, 2])

In [58]:
out.reshape(-1).size()

fake_tick.size()
t1 = torch.nn.Conv2d(time_scales, 9, (5, 3))
t1(fake_tick.unsqueeze(0)).view(-1).size()


torch.Size([432])

In [14]:
kernel_h = 6 # segment_range
kernel_w = 3 # metric_kinds

cons = torch.nn.Sequential(
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU(),
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU(),
    torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
    torch.nn.ReLU()
)

cons(fake_tick.unsqueeze(0)).size()
#fake_tick.unsqueeze(0).size()
in_w = len(cons(fake_tick.unsqueeze(0)).view(-1))

lins = torch.nn.Sequential(
    torch.nn.Linear(in_w, in_w),
    torch.nn.Linear(in_w, in_w),
    torch.nn.Linear(in_w, action_count),
)

def forward(x):
    con_out = cons(x.unsqueeze(0))
    lin_out = lins(con_out.view(-1))
    return lin_out, torch.nn.functional.softmax(lin_out)

forward(fake_tick)

/var/folders/zp/br0t0rj942x005pnp5ghxfy40000gn/T/ipykernel_37709/4049721819.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return lin_out, torch.nn.functional.softmax(lin_out)


(tensor([ 0.0386, -0.0391,  0.0088, -0.0126], grad_fn=<AddBackward0>),
 tensor([0.2600, 0.2406, 0.2524, 0.2470], grad_fn=<SoftmaxBackward>))

In [15]:
t1 = torch.tensor([0.1, 0.4, 0.3, 0.5])
torch.argmax(t1)

tensor(3)

In [16]:
r1 = torch.tensor([[[7.0, 22.0, 14.0, 29.0, 49.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 786.0, 430.0, 3471.0, 3994.0, 960782.0], [7.0, 22.0, 14.0, 29.0, 50.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 786.0, 433.0, 3471.0, 3994.0, 960782.0], [7.0, 22.0, 14.0, 29.0, 50.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 789.0, 430.0, 3471.0, 3994.0, 960786.0], [7.0, 22.0, 14.0, 29.0, 51.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 790.0, 428.0, 3471.0, 3994.0, 960788.0], [7.0, 22.0, 14.0, 29.0, 51.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 811.0, 86.0, 3471.0, 3994.0, 960788.0], [7.0, 22.0, 14.0, 29.0, 52.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 812.0, 63.0, 3471.0, 3994.0, 960788.0], [7.0, 22.0, 14.0, 29.0, 52.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 802.0, 65.0, 3471.0, 3994.0, 960798.0], [7.0, 22.0, 14.0, 29.0, 53.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 803.0, 66.0, 3471.0, 3994.0, 960798.0], [7.0, 22.0, 14.0, 29.0, 53.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 813.0, 66.0, 3471.0, 3994.0, 960799.0], [7.0, 22.0, 14.0, 29.0, 54.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 817.0, 417.0, 3471.0, 3994.0, 960814.0], [7.0, 22.0, 14.0, 29.0, 54.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 795.0, 441.0, 3471.0, 3994.0, 960817.0], [7.0, 22.0, 14.0, 29.0, 55.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 795.0, 425.0, 3471.0, 3994.0, 960817.0], [7.0, 22.0, 14.0, 29.0, 55.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 799.0, 565.0, 3471.0, 3994.0, 960817.0], [7.0, 22.0, 14.0, 29.0, 56.0, 3.0, 3790.000000000001, 3790.000000000001, 3790.000000000001, 3790.000000000001, 795.0, 567.0, 3471.0, 3994.0, 960821.0], [7.0, 22.0, 14.0, 29.0, 56.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 795.0, 568.0, 3471.0, 3994.0, 960822.0], [7.0, 22.0, 14.0, 29.0, 57.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 795.0, 568.0, 3471.0, 3994.0, 960822.0], [7.0, 22.0, 14.0, 29.0, 58.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 795.0, 568.0, 3471.0, 3994.0, 960822.0], [7.0, 22.0, 14.0, 29.0, 58.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 815.0, 575.0, 3471.0, 3994.0, 960822.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 809.0, 594.0, 3471.0, 3994.0, 960829.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3791.0, 3791.0, 3791.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]], [[7.0, 22.0, 14.0, 10.0, 59.0, 3.0, 3794.0, 3795.000000000001, 3793.0, 3794.000000000001, 369.0, 170.0, 3471.0, 3994.0, 923444.0], [7.0, 22.0, 14.0, 11.0, 59.0, 3.0, 3794.000000000001, 3795.0, 3794.0, 3795.0, 261.0, 165.0, 3471.0, 3994.0, 924477.0], [7.0, 22.0, 14.0, 12.0, 59.0, 3.0, 3795.0, 3796.0, 3794.0, 3795.0, 90.0, 1072.0, 3471.0, 3994.0, 924883.0], [7.0, 22.0, 14.0, 13.0, 59.0, 3.0, 3796.0, 3797.000000000001, 3794.0, 3796.0, 402.0, 977.0, 3471.0, 3994.0, 926348.0], [7.0, 22.0, 14.0, 14.0, 59.0, 3.0, 3796.0, 3797.0, 3795.0, 3797.0, 55.0, 837.0, 3471.0, 3994.0, 927739.0], [7.0, 22.0, 14.0, 15.0, 59.0, 3.0, 3797.0, 3797.000000000001, 3795.0, 3797.0, 288.0, 826.0, 3471.0, 3994.0, 928454.0], [7.0, 22.0, 14.0, 16.0, 59.0, 3.0, 3796.000000000001, 3797.0, 3796.0, 3797.0, 177.0, 652.0, 3471.0, 3994.0, 929037.0], [7.0, 22.0, 14.0, 17.0, 59.0, 3.0, 3797.0, 3798.0, 3796.0, 3798.0, 5.0, 1030.0, 3471.0, 3994.0, 930132.0], [7.0, 22.0, 14.0, 18.0, 59.0, 3.0, 3797.0, 3798.0, 3796.0, 3798.0, 403.0, 359.0, 3471.0, 3994.0, 931980.0], [7.0, 22.0, 14.0, 19.0, 59.0, 3.0, 3798.0, 3799.000000000001, 3797.0, 3799.0, 468.0, 582.0, 3471.0, 3994.0, 933383.0], [7.0, 22.0, 14.0, 20.0, 59.0, 3.0, 3799.0, 3799.000000000001, 3797.0, 3799.0, 81.0, 413.0, 3471.0, 3994.0, 935689.0], [7.0, 22.0, 14.0, 21.0, 59.0, 3.0, 3799.0, 3800.0, 3797.0, 3798.0, 325.0, 26.0, 3471.0, 3994.0, 937973.0], [7.0, 22.0, 14.0, 22.0, 59.0, 3.0, 3798.0, 3799.0, 3797.0, 3797.0, 314.0, 418.0, 3471.0, 3994.0, 938826.0], [7.0, 22.0, 14.0, 23.0, 59.0, 3.0, 3797.0, 3798.0, 3794.0, 3794.0, 488.0, 11.0, 3471.0, 3994.0, 940542.0], [7.0, 22.0, 14.0, 24.0, 59.0, 3.0, 3794.0, 3794.000000000001, 3792.0, 3794.000000000001, 203.0, 609.0, 3471.0, 3994.0, 942662.0], [7.0, 22.0, 14.0, 25.0, 59.0, 3.0, 3794.000000000001, 3794.000000000001, 3788.0, 3790.0, 278.0, 125.0, 3471.0, 3994.0, 949933.0], [7.0, 22.0, 14.0, 26.0, 59.0, 3.0, 3790.0, 3791.0, 3789.0, 3790.0, 243.0, 575.0, 3471.0, 3994.0, 952140.0], [7.0, 22.0, 14.0, 27.0, 59.0, 3.0, 3790.0, 3790.0, 3787.0, 3788.0, 101.0, 296.0, 3471.0, 3994.0, 955588.0], [7.0, 22.0, 14.0, 28.0, 59.0, 3.0, 3788.0, 3791.0, 3788.0, 3790.0, 594.0, 375.0, 3471.0, 3994.0, 958904.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3790.0, 3792.000000000001, 3790.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]], [[7.0, 22.0, 10.0, 54.0, 59.0, 3.0, 3783.0, 3791.000000000001, 3782.0, 3790.0, 230.0, 381.0, 3471.0, 3994.0, 798760.0], [7.0, 22.0, 10.0, 59.0, 59.0, 3.0, 3790.0, 3793.0, 3788.0, 3791.0, 58.0, 379.0, 3471.0, 3994.0, 808204.0], [7.0, 22.0, 11.0, 4.0, 59.0, 3.0, 3791.0, 3792.0, 3788.0, 3792.0, 207.0, 593.0, 3471.0, 3994.0, 814095.0], [7.0, 22.0, 11.0, 9.0, 59.0, 3.0, 3792.0, 3794.0, 3791.0, 3792.0, 381.0, 335.0, 3471.0, 3994.0, 819017.0], [7.0, 22.0, 11.0, 14.0, 59.0, 3.0, 3791.0, 3802.0, 3791.0, 3796.0, 491.0, 197.0, 3471.0, 3994.0, 839485.0], [7.0, 22.0, 11.0, 19.0, 59.0, 3.0, 3796.0, 3799.0, 3793.0, 3797.0, 228.0, 213.0, 3471.0, 3994.0, 845922.0], [7.0, 22.0, 11.0, 24.0, 59.0, 3.0, 3797.0, 3802.0, 3797.0, 3801.0, 386.0, 39.0, 3471.0, 3994.0, 853868.0], [7.0, 22.0, 11.0, 29.0, 59.0, 3.0, 3801.0, 3801.000000000001, 3795.0, 3798.0, 13.0, 21.0, 3471.0, 3994.0, 859603.0], [7.0, 22.0, 13.0, 34.0, 59.0, 3.0, 3793.0, 3795.000000000001, 3787.0, 3793.0, 199.0, 215.0, 3471.0, 3994.0, 876786.0], [7.0, 22.0, 13.0, 39.0, 59.0, 3.0, 3793.0, 3799.0, 3792.0, 3796.0, 429.0, 16.0, 3471.0, 3994.0, 884447.0], [7.0, 22.0, 13.0, 44.0, 59.0, 3.0, 3795.0, 3800.0, 3789.0, 3792.0, 394.0, 273.0, 3471.0, 3994.0, 898271.0], [7.0, 22.0, 13.0, 49.0, 59.0, 3.0, 3792.0, 3795.000000000001, 3791.0, 3794.0, 302.0, 229.0, 3471.0, 3994.0, 901828.0], [7.0, 22.0, 13.0, 54.0, 59.0, 3.0, 3793.0, 3795.000000000001, 3790.0, 3791.0, 86.0, 210.0, 3471.0, 3994.0, 908915.0], [7.0, 22.0, 13.0, 59.0, 59.0, 3.0, 3791.0, 3795.000000000001, 3791.0, 3793.0, 189.0, 242.0, 3471.0, 3994.0, 912888.0], [7.0, 22.0, 14.0, 4.0, 59.0, 3.0, 3794.0, 3796.000000000001, 3792.0, 3794.0, 496.0, 133.0, 3471.0, 3994.0, 917873.0], [7.0, 22.0, 14.0, 9.0, 59.0, 3.0, 3794.0, 3796.0, 3792.0, 3794.0, 875.0, 335.0, 3471.0, 3994.0, 922538.0], [7.0, 22.0, 14.0, 14.0, 59.0, 3.0, 3794.0, 3797.000000000001, 3793.0, 3797.0, 55.0, 837.0, 3471.0, 3994.0, 927739.0], [7.0, 22.0, 14.0, 19.0, 59.0, 3.0, 3797.0, 3799.000000000001, 3795.0, 3799.0, 468.0, 582.0, 3471.0, 3994.0, 933383.0], [7.0, 22.0, 14.0, 24.0, 59.0, 3.0, 3799.0, 3800.0, 3792.0, 3794.000000000001, 203.0, 609.0, 3471.0, 3994.0, 942662.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3794.000000000001, 3794.000000000001, 3787.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]], [[7.0, 22.0, 21.0, 29.0, 59.0, 3.0, 3790.0, 3792.000000000001, 3784.0, 3789.0, 491.0, 18.0, 3471.0, 3994.0, 239962.0], [7.0, 22.0, 21.0, 44.0, 59.0, 3.0, 3789.0, 3791.0, 3773.0, 3778.000000000001, 177.0, 398.0, 3471.0, 3994.0, 293251.0], [7.0, 22.0, 21.0, 59.0, 59.0, 3.0, 3778.000000000001, 3792.0, 3777.0, 3790.0, 392.0, 552.0, 3471.0, 3994.0, 344569.0], [7.0, 22.0, 22.0, 14.0, 59.0, 3.0, 3789.0, 3792.000000000001, 3787.0, 3791.0, 437.0, 414.0, 3471.0, 3994.0, 367645.0], [7.0, 22.0, 22.0, 29.0, 59.0, 3.0, 3791.0, 3795.000000000001, 3785.0, 3786.0, 307.0, 284.0, 3471.0, 3994.0, 400256.0], [7.0, 22.0, 22.0, 44.0, 59.0, 3.0, 3786.0, 3792.000000000001, 3786.0, 3790.0, 44.0, 884.0, 3471.0, 3994.0, 418239.0], [7.0, 22.0, 22.0, 59.0, 59.0, 3.0, 3790.0, 3794.0, 3787.0, 3790.0, 11.0, 15.0, 3471.0, 3994.0, 442464.0], [7.0, 22.0, 9.0, 14.0, 59.0, 3.0, 3794.0, 3816.0, 3790.0, 3806.0, 506.0, 65.0, 3471.0, 3994.0, 561347.0], [7.0, 22.0, 9.0, 29.0, 59.0, 3.0, 3807.0, 3808.0, 3792.0, 3801.0, 460.0, 360.0, 3471.0, 3994.0, 622785.0], [7.0, 22.0, 9.0, 44.0, 59.0, 3.0, 3801.0, 3803.000000000001, 3794.0, 3801.0, 27.0, 319.0, 3471.0, 3994.0, 661898.0], [7.0, 22.0, 9.0, 59.0, 59.0, 3.0, 3800.0, 3802.000000000001, 3794.0, 3797.0, 24.0, 358.0, 3471.0, 3994.0, 682452.0], [7.0, 22.0, 10.0, 14.0, 59.0, 3.0, 3797.0, 3800.000000000001, 3794.0, 3795.0, 125.0, 320.0, 3471.0, 3994.0, 703059.0], [7.0, 22.0, 10.0, 44.0, 59.0, 3.0, 3796.0, 3797.000000000001, 3782.0, 3787.0, 177.0, 321.0, 3471.0, 3994.0, 775762.0], [7.0, 22.0, 10.0, 59.0, 59.0, 3.0, 3787.0, 3793.0, 3782.0, 3791.0, 58.0, 379.0, 3471.0, 3994.0, 808204.0], [7.0, 22.0, 11.0, 14.0, 59.0, 3.0, 3791.0, 3802.0, 3788.0, 3796.0, 491.0, 197.0, 3471.0, 3994.0, 839485.0], [7.0, 22.0, 11.0, 29.0, 59.0, 3.0, 3796.0, 3802.0, 3793.0, 3798.0, 13.0, 21.0, 3471.0, 3994.0, 859603.0], [7.0, 22.0, 13.0, 44.0, 59.0, 3.0, 3793.0, 3800.0, 3787.0, 3792.0, 394.0, 273.0, 3471.0, 3994.0, 898271.0], [7.0, 22.0, 13.0, 59.0, 59.0, 3.0, 3792.0, 3795.000000000001, 3790.0, 3793.0, 189.0, 242.0, 3471.0, 3994.0, 912888.0], [7.0, 22.0, 14.0, 14.0, 59.0, 3.0, 3794.0, 3797.000000000001, 3792.0, 3797.0, 55.0, 837.0, 3471.0, 3994.0, 927739.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3797.0, 3800.0, 3787.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]], [[7.0, 20.0, 22.0, 59.0, 59.0, 1.0, 3744.0, 3749.0, 3723.0, 3734.0, 37.0, 171.0, 3451.0, 3970.0, 391180.0], [7.0, 20.0, 9.0, 59.0, 59.0, 1.0, 3735.0, 3735.0, 3708.0, 3709.0, 550.0, 595.0, 3451.0, 3970.0, 593648.0], [7.0, 20.0, 10.0, 59.0, 59.0, 1.0, 3708.000000000001, 3718.0000000000005, 3701.0, 3716.0, 205.0, 812.0, 3451.0, 3970.0, 713200.0], [7.0, 20.0, 11.0, 29.0, 59.0, 1.0, 3716.0, 3717.000000000001, 3701.0, 3701.0, 758.0, 177.0, 3451.0, 3970.0, 790258.0], [7.0, 20.0, 13.0, 59.0, 59.0, 1.0, 3701.0000000000005, 3711.0, 3696.000000000001, 3701.0, 7.0, 501.0, 3451.0, 3970.0, 893068.0], [7.0, 20.0, 14.0, 59.0, 59.0, 1.0, 3700.0, 3712.0, 3698.0, 3702.000000000001, 75.0, 40.0, 3451.0, 3970.0, 1026156.0], [7.0, 21.0, 21.0, 59.0, 59.0, 2.0, 3699.0, 3712.0, 3697.0, 3705.0, 102.0, 634.0, 3458.0, 3979.0, 142261.0], [7.0, 21.0, 22.0, 59.0, 59.0, 2.0, 3705.0, 3720.000000000001, 3705.0, 3716.0, 919.0, 198.0, 3458.0, 3979.0, 271396.0], [7.0, 21.0, 9.0, 59.0, 59.0, 2.0, 3716.0, 3745.000000000001, 3716.0, 3738.0, 304.0, 161.0, 3458.0, 3979.0, 579003.0], [7.0, 21.0, 10.0, 59.0, 59.0, 2.0, 3738.0, 3741.000000000001, 3728.0, 3733.0, 257.0, 71.0, 3458.0, 3979.0, 658066.0], [7.0, 21.0, 11.0, 29.0, 59.0, 2.0, 3733.0, 3739.000000000001, 3731.0, 3735.0, 136.0, 329.0, 3458.0, 3979.0, 692400.0], [7.0, 21.0, 13.0, 59.0, 59.0, 2.0, 3733.0, 3743.000000000001, 3731.0, 3741.0, 318.0, 753.0, 3458.0, 3979.0, 740114.0], [7.0, 21.0, 14.0, 59.0, 59.0, 2.0, 3742.0, 3765.000000000001, 3739.0, 3764.000000000001, 323.0, 390.0, 3458.0, 3979.0, 1025122.0], [7.0, 22.0, 21.0, 59.0, 59.0, 3.0, 3768.0, 3795.0, 3768.0, 3790.0, 392.0, 552.0, 3471.0, 3994.0, 344569.0], [7.0, 22.0, 22.0, 59.0, 59.0, 3.0, 3789.0, 3795.000000000001, 3785.0, 3790.0, 11.0, 15.0, 3471.0, 3994.0, 442464.0], [7.0, 22.0, 9.0, 59.0, 59.0, 3.0, 3794.0, 3816.0, 3790.0, 3797.0, 24.0, 358.0, 3471.0, 3994.0, 682452.0], [7.0, 22.0, 10.0, 59.0, 59.0, 3.0, 3797.0, 3800.000000000001, 3782.0, 3791.0, 58.0, 379.0, 3471.0, 3994.0, 808204.0], [7.0, 22.0, 11.0, 29.0, 59.0, 3.0, 3791.0, 3802.0, 3788.0, 3798.0, 13.0, 21.0, 3471.0, 3994.0, 859603.0], [7.0, 22.0, 13.0, 59.0, 59.0, 3.0, 3793.0, 3800.0, 3787.0, 3793.0, 189.0, 242.0, 3471.0, 3994.0, 912888.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3794.0, 3800.0, 3787.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [7.0, 1.0, 14.0, 59.0, 59.0, 3.0, 3566.0000000000005, 3574.0000000000005, 3542.0, 3567.0, 27.0, 685.0, 3318.0, 3817.0, 1086112.0], [7.0, 2.0, 14.0, 59.0, 59.0, 4.0, 3566.0000000000005, 3581.0000000000005, 3560.0, 3574.0, 46.0, 403.0, 3311.0, 3810.0, 809867.0], [7.0, 3.0, 14.0, 59.0, 59.0, 5.0, 3577.0, 3620.0000000000005, 3574.0, 3619.0, 618.0, 63.0, 3319.0, 3818.0, 1233967.0], [7.0, 6.0, 14.0, 59.0, 59.0, 1.0, 3622.0, 3629.0000000000005, 3603.0, 3618.0, 27.0, 38.0, 3350.0, 3855.0, 949955.0], [7.0, 7.0, 14.0, 59.0, 59.0, 2.0, 3619.0, 3654.0, 3619.0, 3634.0, 32.0, 311.0, 3361.0, 3868.0, 957200.0], [7.0, 8.0, 14.0, 59.0, 59.0, 3.0, 3638.0, 3705.000000000001, 3636.0, 3700.0, 265.0, 7.0, 3381.0, 3890.0, 1368038.0], [7.0, 9.0, 14.0, 59.0, 59.0, 4.0, 3708.0, 3742.000000000001, 3703.0, 3725.0, 40.0, 8.0, 3420.0, 3935.0, 1172490.0], [7.0, 10.0, 14.0, 59.0, 59.0, 5.0, 3723.0, 3723.0, 3680.0, 3690.0, 412.0, 7.0, 3459.0, 3980.0, 1178729.0], [7.0, 13.0, 14.0, 59.0, 59.0, 1.0, 3692.0, 3768.0, 3692.0, 3739.0, 76.0, 364.0, 3443.0, 3962.0, 1377089.0], [7.0, 14.0, 14.0, 59.0, 59.0, 2.0, 3741.0, 3754.000000000001, 3716.0, 3744.0, 67.0, 59.0, 3473.0, 3996.0, 953880.0], [7.0, 15.0, 14.0, 59.0, 59.0, 3.0, 3743.0, 3762.0, 3721.0, 3746.0, 7.0, 5.0, 3473.0, 3996.0, 858731.0], [7.0, 16.0, 14.0, 59.0, 59.0, 4.0, 3739.0, 3743.000000000001, 3692.0, 3702.0, 28.0, 192.0, 3477.0, 4000.0, 1255797.0], [7.0, 17.0, 14.0, 59.0, 59.0, 5.0, 3706.0, 3734.000000000001, 3686.0, 3726.0, 1859.0, 24.0, 3458.0, 3979.0, 926244.0], [7.0, 20.0, 14.0, 59.0, 59.0, 1.0, 3721.0, 3749.0, 3696.000000000001, 3702.000000000001, 75.0, 40.0, 3451.0, 3970.0, 1026156.0], [7.0, 21.0, 14.0, 59.0, 59.0, 2.0, 3699.0, 3765.000000000001, 3697.0, 3764.000000000001, 323.0, 390.0, 3458.0, 3979.0, 1025122.0], [7.0, 22.0, 14.0, 29.0, 59.0, 3.0, 3768.0, 3816.0, 3768.0, 3791.0, 810.0, 594.0, 3471.0, 3994.0, 960829.0]]])
r1.size()

torch.Size([6, 20, 15])

In [17]:
forward(r1)

/var/folders/zp/br0t0rj942x005pnp5ghxfy40000gn/T/ipykernel_37709/4049721819.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return lin_out, torch.nn.functional.softmax(lin_out)


(tensor([-2011.3035,  -725.1359, -4176.2686,  1715.6284],
        grad_fn=<AddBackward0>),
 tensor([0., 0., 0., 1.], grad_fn=<SoftmaxBackward>))

In [25]:
import torch


class TestNN(torch.nn.Module):
    def __init__(self):
        super(TestNN, self).__init__()
        self.cons = torch.nn.Sequential(
            torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(time_scales, time_scales, (kernel_h, kernel_w), padding=1),
            torch.nn.ReLU())
        #cons(fake_tick.unsqueeze(0)).size()
        #fake_tick.unsqueeze(0).size()
        in_w = len(cons(fake_tick.unsqueeze(0)).view(-1))
        self.lins = torch.nn.Sequential(
            torch.nn.Linear(in_w, in_w),
            torch.nn.Linear(in_w, in_w),
            torch.nn.Linear(in_w, action_count))
    def forward(self, x):
        con_out = self.cons(x.unsqueeze(0))
        lin_out = self.lins(con_out.view(-1))
        return lin_out, torch.nn.functional.softmax(lin_out)

tt = TestNN()
torch.onnx.export(tt, r1, "test.onnx", opset_version=11)

/var/folders/zp/br0t0rj942x005pnp5ghxfy40000gn/T/ipykernel_37709/3688773127.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return lin_out, torch.nn.functional.softmax(lin_out)
